In [7]:
from pathlib import Path
import pandas as pd
from prefect import flow, task
from prefect_gcp.cloud_storage import GcsBucket
from random import randint
import os
from prefect.deployments import Deployment
from prefect.infrastructure.docker import DockerContainer
from prefect_gcp import GcpCredentials


## Question 1. Load January 2020 data

Using the `etl_web_to_gcs.py` flow that loads taxi data into GCS as a guide, create a flow that loads the green taxi CSV dataset for January 2020 into GCS and run it. Look at the logs to find out how many rows the dataset has.

How many rows does that dataset have?

* 447,770
* 766,792
* 299,234
* 822,132

In [4]:
@task(retries=3)
def fetch(dataset_url: str) -> pd.DataFrame:
    """Read taxi data from web into pandas DataFrame"""
    # if randint(0, 1) > 0:
    #     raise Exception

    df = pd.read_csv(dataset_url)
    return df


@task(log_prints=True)
def clean(df: pd.DataFrame, color: str) -> pd.DataFrame:
    """Fix dtype issues"""

    if color == "green":
        df["lpep_pickup_datetime"] = pd.to_datetime(df["lpep_pickup_datetime"])
        df["lpep_dropoff_datetime"] = pd.to_datetime(df["lpep_dropoff_datetime"])
        print(df.head(2))
        print(f"columns: {df.dtypes}")
        print(f"rows: {len(df)}")
        return df
    elif(color == "yellow"):
        df["tpep_pickup_datetime"] = pd.to_datetime(df["tpep_pickup_datetime"])
        df["tpep_dropoff_datetime"] = pd.to_datetime(df["tpep_dropoff_datetime"])
        print(df.head(2))
        print(f"columns: {df.dtypes}")
        print(f"rows: {len(df)}")
        return df

@task()
def write_local(df: pd.DataFrame, color: str, dataset_file: str) -> Path:
    """Write DataFrame out locally as parquet file"""
    mydir = (f"data/{color}/")
    check_folder = os.path.isdir(mydir)
    
    if not check_folder:
        os.makedirs(mydir)
        print("created folder:", mydir)
    else:
        print(mydir, "Folder already exists")
        
    path = Path(f"data/{color}/{dataset_file}.parquet")
    df.to_parquet(path, compression="gzip")
    return path


@task()
def write_gcs(path: Path) -> None:
    """Upload local parquet file to GCS"""
    gcs_block = GcsBucket.load("zoom-gcs")
    gcs_block.upload_from_path(from_path=path, to_path=path)
    return


@flow()
def etl_web_to_gcs(months: list[int] = [1], year: int = 2020, color: str = "green") -> None:
    """The main ETL function"""

    for month in months:
        dataset_file = f"{color}_tripdata_{year}-{month:02}"
        dataset_url = f"https://github.com/DataTalksClub/nyc-tlc-data/releases/download/{color}/{dataset_file}.csv.gz"

        df = fetch(dataset_url)
        df_clean = clean(df, color)
        path = write_local(df_clean, color, dataset_file)
        write_gcs(path)

if __name__ == "__main__":
    etl_web_to_gcs()

/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/tasks.py:256: UserWarning: A task named 'fetch' and defined at '/var/folders/lf/203pfq2545v143_jyxtx1sgh0000gn/T/ipykernel_12638/24667178.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/tasks.py:256: UserWarning: A task named 'clean' and defined at '/var/folders/lf/203pfq2545v143_jyxtx1sgh0000gn/T/ipykernel_12638/24667178.py:11' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/tasks.py:256: UserWarning: A task named 'write_local' and defined at '/var/folders/lf/203pfq2545v143_jyxtx1sgh0000gn/T/ipykernel_12638/24667178.py:30' conflicts with another task

18:06:53.383 | INFO    | prefect.engine - Created flow run 'refreshing-lion' for flow 'etl-web-to-gcs'

18:06:53.504 | INFO    | Flow run 'refreshing-lion' - Created task run 'fetch-b4598a4a-0' for task 'fetch'

18:06:53.505 | INFO    | Flow run 'refreshing-lion' - Executing 'fetch-b4598a4a-0' immediately...

/var/folders/lf/203pfq2545v143_jyxtx1sgh0000gn/T/ipykernel_12638/24667178.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dataset_url)


...

18:06:57.330 | INFO    | Flow run 'refreshing-lion' - Executing 'write_gcs-1145c921-0' immediately...

18:06:57.507 | INFO    | Task run 'write_gcs-1145c921-0' - Getting bucket 'dtc_data_lake_esoteric-pen-376110'.

18:06:57.948 | INFO    | Task run 'write_gcs-1145c921-0' - Uploading from PosixPath('data/green/green_tripdata_2020-01.parquet') to the bucket 'dtc_data_lake_esoteric-pen-376110' path 'data/green/green_tripdata_2020-01.parquet'.

18:06:59.024 | INFO    | Task run 'write_gcs-1145c921-0' - Finished in state Completed()

18:06:59.053 | INFO    | Flow run 'refreshing-lion' - Finished in state Completed('All states completed.')

# Ans = 447770

# Cron
## Question 2. Scheduling with Cron

Cron is a common scheduling specification for workflows. 

Using the flow in `etl_web_to_gcs.py`, create a deployment to run on the first of every month at 5am UTC. What’s the cron schedule for that?

- `0 5 1 * *`
- `0 0 5 1 *`
- `5 * 1 0 *`
- `* * 5 1 0`

![Alt text](images/cron_v.png "Cron")
![Alt text](images/cron_u.png "Cron")

Courtesy of https://crontab.guru/#0_5_1_1-12_*

# Ans = `0 5 1 * *`

## Question 3. Loading data to BigQuery 

Using `etl_gcs_to_bq.py` as a starting point, modify the script for extracting data from GCS and loading it into BigQuery. This new script should not fill or remove rows with missing values. (The script is really just doing the E and L parts of ETL).

The main flow should print the total number of rows processed by the script. Set the flow decorator to log the print statement.

Parametrize the entrypoint flow to accept a list of months, a year, and a taxi color. 

Make any other necessary changes to the code for it to function as required.

Create a deployment for this flow to run in a local subprocess with local flow code storage (the defaults).

Make sure you have the parquet data files for Yellow taxi data for Feb. 2019 and March 2019 loaded in GCS. Run your deployment to append this data to your BiqQuery table. How many rows did your flow code process?

- 14,851,920
- 12,282,990
- 27,235,753
- 11,338,483

In [5]:
etl_web_to_gcs(months =[2,3], year=2019, color="yellow")

18:07:10.727 | INFO    | prefect.engine - Created flow run 'fair-bullfinch' for flow 'etl-web-to-gcs'

18:07:10.891 | INFO    | Flow run 'fair-bullfinch' - Created task run 'fetch-b4598a4a-0' for task 'fetch'

18:07:10.893 | INFO    | Flow run 'fair-bullfinch' - Executing 'fetch-b4598a4a-0' immediately...

18:07:43.482 | INFO    | Task run 'fetch-b4598a4a-0' - Finished in state Completed()

18:07:43.511 | INFO    | Flow run 'fair-bullfinch' - Created task run 'clean-b9fd7e03-0' for task 'clean'

...

18:09:19.444 | INFO    | Task run 'write_gcs-1145c921-1' - Getting bucket 'dtc_data_lake_esoteric-pen-376110'.

18:09:19.731 | INFO    | Task run 'write_gcs-1145c921-1' - Uploading from PosixPath('data/yellow/yellow_tripdata_2019-03.parquet') to the bucket 'dtc_data_lake_esoteric-pen-376110' path 'data/yellow/yellow_tripdata_2019-03.parquet'.

18:09:33.896 | INFO    | Task run 'write_gcs-1145c921-1' - Finished in state Completed()

18:09:33.926 | INFO    | Flow run 'fair-bullfinch' - Finished in state Completed('All states completed.')

[Completed(message=None, type=COMPLETED, result=         VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
 0               1  2019-02-01 00:59:04   2019-02-01 01:07:27                1   
 1               1  2019-02-01 00:33:09   2019-02-01 01:03:58                1   
 2               1  2019-02-01 00:09:03   2019-02-01 00:09:16                1   
 3               1  2019-02-01 00:45:38   2019-02-01 00:51:10                1   
 4               1  2019-02-01 00:25:30   2019-02-01 00:28:14                1   
 ...           ...                  ...                   ...              ...   
 7019370         2  2019-02-28 23:29:08   2019-02-28 23:29:11                1   
 7019371         2  2019-02-28 22:48:47   2019-02-28 23:50:19                1   
 7019372         2  2019-02-28 23:41:23   2019-02-28 23:42:23                1   
 7019373         2  2019-02-28 23:12:52   2019-02-28 23:14:16                1   
 7019374         2  2019-02-28 23:10:35   2019-02-2

In [12]:
@task(retries=3)
def extract_from_gcs(color: str, year: int, month: int) -> Path:
    """Download trip data from GCS"""
    gcs_path = f"data/{color}/{color}_tripdata_{year}-{month:02}.parquet"
    gcs_block = GcsBucket.load("zoom-gcs")
    gcs_block.get_directory(from_path=gcs_path, local_path=f"gcp_data/")
    return Path(gcs_path)


# @task()
# def transform(path: Path) -> pd.DataFrame:
#     """Data cleaning example"""
#     df = pd.read_parquet(path)
#     print(f"pre: missing passenger count: {df['passenger_count'].isna().sum()}")
#     df["passenger_count"].fillna(0, inplace=True)
#     print(f"post: missing passenger count: {df['passenger_count'].isna().sum()}")
#     return df


@task()
def write_bq(df: pd.DataFrame) -> None:
    """Write DataFrame to BiqQuery"""

    gcp_credentials_block = GcpCredentials.load("zoom-gcp-creds")

    df.to_gbq(
        destination_table="dezoomcamp.rides",
        project_id="esoteric-pen-376110",
        credentials=gcp_credentials_block.get_credentials_from_service_account(),
        chunksize=500_000,
        if_exists="append",
    )

@flow(log_prints=True)
def etl_gcs_to_bq( months: list[int] = [2, 3], year: int = 2019, color: str = "yellow"):
    """Main ETL flow to load data into Big Query"""
    # color = "yellow"
    # year = 2021
    # month = 1
    no_of_rows = 0

    for month in months:
        
        path = extract_from_gcs(color, year, month)
        
        df = pd.read_parquet(path)
        # df = transform(path)
        print(f"rows: {len(df)}")
        no_of_rows+=len(df)
        write_bq(df)

    print("Total number of rows processed:", no_of_rows)

if __name__ == "__main__":
    
    etl_gcs_to_bq()
    # dep = Deployment.build_from_flow(
    # flow=etl_gcs_to_bq,
    # name="gcs_to_bq"
    # )

    # dep.apply()

/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/tasks.py:256: UserWarning: A task named 'extract_from_gcs' and defined at '/var/folders/lf/203pfq2545v143_jyxtx1sgh0000gn/T/ipykernel_16243/1533059540.py:1' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/tasks.py:256: UserWarning: A task named 'write_bq' and defined at '/var/folders/lf/203pfq2545v143_jyxtx1sgh0000gn/T/ipykernel_16243/1533059540.py:20' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/Users/manojkl/miniforge3/envs/etl_gcp/lib/python3.9/site-packages/prefect/flows.py:206: UserWarning: A flow named 'etl-gcs-to-bq' and defined at '/var/folders/lf/203pfq2545v143_jyxtx1sgh0000gn/T/ipykernel_16243/1533059540.py:34' confl

20:23:37.485 | INFO    | prefect.engine - Created flow run 'attentive-hog' for flow 'etl-gcs-to-bq'

20:23:37.545 | INFO    | Flow run 'attentive-hog' - Created task run 'extract_from_gcs-968e3b65-0' for task 'extract_from_gcs'

20:23:37.546 | INFO    | Flow run 'attentive-hog' - Executing 'extract_from_gcs-968e3b65-0' immediately...

20:23:37.916 | INFO    | Task run 'extract_from_gcs-968e3b65-0' - Downloading blob named data/yellow/yellow_tripdata_2019-02.parquet from the dtc_data_lake_esoteric-pen-376110 bucket to gcp_data/data/yellow/yellow_tripdata_2019-02.parquet

...

20:24:54.215 | INFO    | Flow run 'attentive-hog' - rows: 7832545

20:24:54.234 | INFO    | Flow run 'attentive-hog' - Created task run 'write_bq-b366772c-1' for task 'write_bq'

20:24:54.235 | INFO    | Flow run 'attentive-hog' - Executing 'write_bq-b366772c-1' immediately...

20:25:59.816 | INFO    | Task run 'write_bq-b366772c-1' - Finished in state Completed()

20:25:59.831 | INFO    | Flow run 'attentive-hog' - Finished in state Completed('All states completed.')